<a href="https://colab.research.google.com/github/chanyoung98/-/blob/main/2%EC%9E%A5_%EB%8C%80%EB%A6%AC%EC%A0%90%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B0%80%EA%B3%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import pandas as pd
uriage_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/파이썬실무테크닉/2장-대리점 데이터 가공/uriage.csv')
uriage_data.head(3)

#데이터의 정합성에 문제 확인

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬


In [34]:
kokyaku_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/파이썬실무테크닉/2장-대리점 데이터 가공/kokyaku_daicho.xlsx')
kokyaku_data.head(3)

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00


정합성을 갖추기 위해서는 먼저 데이터의 속성이나 의미를 이해해야 한다. 정합성 확보는 우선 데이터의 오류를 파악하는 것부터 시작한다.

먼저, 매출 이력에서 item_name을 추출해서 데이터의 오류를 확인해보자.

In [35]:
#오류 데이터
uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [36]:
#결측치 데이터
uriage_data["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

# 데이터에 오류 있는 상태로 집계해보자
먼저 '매출 이력'에서 상품별로 월 매출 합계를 집계해보자.

In [37]:
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index = "purchase_month", columns = "item_name", aggfunc = "size", fill_value = 0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


위의 피벗 테이블을 보면, '상품 S'나 '상품 s'처럼 원래 동일한 상품이 다른 상품으로 집계된 것을 확인할 수 있다.  
또한, 7x99를 통해 데이터의 오류로 인하여 원래 26개의 상품이 99개의 상품으로 늘어났다.  

마찬가지로 가로축에 item_price를 설정해서 집계해보자. 이 또한 집계 결과가 올바르지 않은 것을 확인할 수 있다.

In [38]:
res = uriage_data.pivot_table(index = "purchase_month", columns = "item_name", aggfunc = "sum", fill_value = 0)
res

item_price                                                  \
item_name           상 품 n  상품 E  상품 M  상품 P  상품 S  상품 W  상품 X   상품W 상  품O   
purchase_month                                                              
201901               1400     0     0     0     0     0     0     0     0   
201902                  0     0     0     0     0     0  2400     0     0   
201903                  0   500  1300  1600     0     0     0     0     0   
201904                  0     0     0     0     0     0     0  2300     0   
201905                  0     0     0     0  1900     0     0     0     0   
201906                  0     0     0     0     0  2300     0     0     0   
201907                  0     0     0     0     0     0     0     0     0   

                      ...                                                  \
item_name      상  품Q  ...   상품k   상품l   상품o   상품p   상품r   상품s   상품t   상품v   
purchase_month        ...                                                   
201901             0  ...  1100  1200  1500     0     0     0     0     0   
201902             0  ...     0     0     0     0     0  1900  2000  2200   
201903             0  ...     0     0     0     0     0     0     0     0   
201904          1700  ...     0     0     0     0     0  1900     0     0   
201905             0  ...     0  1200     0     0     0     0     0     0   
201906             0  ...     0     0     0  1600     0     0     0     0   
201907             0  ...     0     0  1500     0  1800     0     0     0   

                            
item_name        상품x   상품y  
purchase_month              
201901             0     0  
201902             0     0  
201903             0     0  
201904             0     0  
201905             0  2500  
201906          2400     0  
201907             0     0  

[7 rows x 99 columns]

이제 데이터의 오류를 수정하는 데이터 가공에 도전해보자.

# 상품명 오류를 수정하자
우선 상품명의 오류를 수정해보자.  

먼저 현황부터 파악한다. 수정 후의 결과가 올바른 결과인지 아닌지를 판정하기 위해 현재 상태를 파악하는 것이 중요하다.

In [39]:
print(len(pd.unique(uriage_data.item_name)))

99


매출 이력의 item_name의 중복을 제외한 데이터 건수를 pd.unique로 확인 가능하다. 우리는 99개의 item_name을 26개로 줄여야한다.

In [40]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace(" ","")

In [41]:
uriage_data.sort_values(by = ['item_name'], ascending = True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [42]:
#데이터 처리 확인 완료
len(uriage_data['item_name'].unique())

26

# 금액의 결측치를 수정하자
금액의 결측치 또한 수정해보자.  

결측치에 따라 현장에서 조사를 거쳐 수정하거나 부서원이 결측치를 채워 넣는 등의 여러가지 대응이 가능하다. 이번 경우는 상품 단가가 집계 중에 변하지 않았기에 프로그램으로 결측치를 수정할 수 있다.  

먼저, 데이터에 결측치가 있는지 확인해보자.

In [43]:
uriage_data.isnull().sum()

purchase_date       0
item_name           0
item_price        387
customer_name       0
purchase_month      0
dtype: int64

집계 중 상품 단가가 바뀌지 않았다는 전제가 있기에, 상품의 단가들로 결측치를 채워보자.

In [44]:
flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()): #아이템 하나 봤을 때
  price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max() #결측치가 아니면서 item_name이 trg인 행, item_price 열,, max()를 왜하는거지? 쨋든 대체 price 뽑음
  uriage_data["item_price"].loc[(flg_is_null) & (uriage_data['item_name'] == trg)] = price #결측치에 넣기
uriage_data.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [45]:
#결측치가 없는지 체크하자
uriage_data.isnull().sum()

purchase_date     0
item_name         0
item_price        0
customer_name     0
purchase_month    0
dtype: int64

이후, 각 상품의 금액이 정상적으로 수정되었는지 확인하자

In [46]:
for trg in list(uriage_data['item_name'].sort_values().unique()): #아이템 목록들 하나씩,, trg로 넘기기
  print(trg + '의 최고가 : ' + str(uriage_data.loc[uriage_data['item_name'] == trg, 'item_price'].max())
        + '의 최저가 : ' + str(uriage_data.loc[uriage_data['item_name'] == trg, 'item_price'].min(skipna = False)))

상품A의 최고가 : 100.0의 최저가 : 100.0
상품B의 최고가 : 200.0의 최저가 : 200.0
상품C의 최고가 : 300.0의 최저가 : 300.0
상품D의 최고가 : 400.0의 최저가 : 400.0
상품E의 최고가 : 500.0의 최저가 : 500.0
상품F의 최고가 : 600.0의 최저가 : 600.0
상품G의 최고가 : 700.0의 최저가 : 700.0
상품H의 최고가 : 800.0의 최저가 : 800.0
상품I의 최고가 : 900.0의 최저가 : 900.0
상품J의 최고가 : 1000.0의 최저가 : 1000.0
상품K의 최고가 : 1100.0의 최저가 : 1100.0
상품L의 최고가 : 1200.0의 최저가 : 1200.0
상품M의 최고가 : 1300.0의 최저가 : 1300.0
상품N의 최고가 : 1400.0의 최저가 : 1400.0
상품O의 최고가 : 1500.0의 최저가 : 1500.0
상품P의 최고가 : 1600.0의 최저가 : 1600.0
상품Q의 최고가 : 1700.0의 최저가 : 1700.0
상품R의 최고가 : 1800.0의 최저가 : 1800.0
상품S의 최고가 : 1900.0의 최저가 : 1900.0
상품T의 최고가 : 2000.0의 최저가 : 2000.0
상품U의 최고가 : 2100.0의 최저가 : 2100.0
상품V의 최고가 : 2200.0의 최저가 : 2200.0
상품W의 최고가 : 2300.0의 최저가 : 2300.0
상품X의 최고가 : 2400.0의 최저가 : 2400.0
상품Y의 최고가 : 2500.0의 최저가 : 2500.0
상품Z의 최고가 : 2600.0의 최저가 : 2600.0


모든 상품의 최대 금액과 최소 금액이 일치하는 것으로 봐서 성공적으로 금액을 수정했다는 것을 알 수 있다.

# 고객 이름의 오류 수정하기  
고객 정보에서 고객 이름의 오류를 수정해보자.

In [47]:
kokyaku_data["고객이름"].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [48]:
uriage_data["customer_name"].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [49]:
#kokyaku_data의 고객 이름 수정하기
kokyaku_data["고객이름"] = kokyaku_data["고객이름"].str.replace(" ","")
kokyaku_data.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04 00:00:00
1,김도윤,E시,42782
2,김지한,A시,2018-01-07 00:00:00
3,김하윤,F시,42872
4,김시온,E시,43127


#날짜 오류를 수정하자  
다음으로 고객 대장의 등록일 오류를 수정해보자. 엑셀 데이터를 취급할 때는 주의해야 할 점으로 '서식이 다른 데이터가 섞여 있을 수 있다.'라는 점을 들 수 있다.  

날짜를 동일한 포맷으로 통일해보자.

In [61]:
#isdigit -> 숫자를 판별
#astype으로 str 타입으로 자료형 변경 후 데이터프레임 자체의 속성을 str로 바꾼다. 이후 isdigit() 함수 사용
flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

22

In [62]:
pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].astype("float"), unit = "D")

1     42782 days
3     42872 days
4     43127 days
21    42920 days
27    42901 days
47    42741 days
49    42929 days
53    42833 days
76    43188 days
80    43110 days
99    42885 days
114   43254 days
118   43129 days
122   43206 days
139   42880 days
143   42818 days
155   42754 days
172   43181 days
179   42743 days
183   42940 days
186   43294 days
192   43259 days
Name: 등록일, dtype: timedelta64[ns]

숫자로 등록되어있는 부분을 수정해보자.

In [60]:
#엑셀에서, 42782를 날짜 형태로 나타내면‘1900/01/01’을 기준으로 42782일을 더한, '2017/02/16’이 된다. (엑셀의 날짜 기억법)
#unit = "D" : 'days'를 의미. ~일 기준으로 변경하라는 뜻
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].astype("float"), unit = "D") + pd.to_datetime("1900/01/01")
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [67]:
#일반적인 날짜 데이터도 서식을 통일해주자
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "등록일"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [79]:
kokyaku_data['등록일'] = pd.concat([fromSerial, fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


등록일이 깨끗하게 수정된 것 확인 가능하다. 등록일로부터 등록월을 추출해서 집계해보자.

In [91]:
kokyaku_data['등록연월'] = kokyaku_data['등록일'].dt.strftime("%Y%m")
rslt = kokyaku_data.groupby("등록연월").count()['고객이름'] #등록연월별로 고객이름 얼마나 있는지 확인
print(rslt)
print(rslt.sum())
print(len(kokyaku_data))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200
200


연월 단위의 집계 결과를 전부 더하면 200이 되기 때문에 날짜의 수정이 정상적으로 된 것도 확인 가능하다. 처음에 확인했던 것 처럼 등록일에 숫자 데이터가 남아있는지 확인해보자.

In [94]:
#처음에 22개였던 결과가 0개가 되어 모든 숫자 데이터가 날짜로 수정되었음을 확인할 수 있다.
kokyaku_data['등록일'].astype(str).str.isdigit().sum()

0

# 고객 이름을 키로 두 개의 데이터를 결합(조인)하자.  
데이터 분석을 위해 매출 이력과 고객 정보를 결합한 데이터를 작성한다.  

정합성이 없는 두 데이터이기에, 두 개의 데이터의 서로 다른 열을 지정해서 결합하자.


In [95]:
kokyaku_data.head()

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-18,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-19,201705
4,김시온,E시,2018-01-29,201801


In [96]:
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [101]:
#left_on, right_on으로 결합할 데이터를 지정.
#how는 결합방법이다. left로 지정함으로써 왼쪽을 기준으로 오른쪽을 결합한다는 뜻.
join_data = pd.merge(uriage_data, kokyaku_data, left_on = "customer_name", right_on = "고객이름", how = 'left')
join_data = join_data.drop("customer_name", axis = 1)
join_data.head()

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706


데이터를 가공함으로써 분석에 적합한 데이터의 형태가 되었다. 이런 데이터 가공을 '데이터 정제'라고 한다.  

# 정제한 데이터를 덤프하자.  
데이터를 수정하여 깔끔하게 정리했지만, 우리의 목적은 '데이터 가공 및 정제'가 아니라 '데이터 분석'이다. 깨끗해진 데이터를 파일로 출력(덤프)해주고, 분석할 때 출력한 파일을 다시 읽어들이면 데이터 정제를 다시 할 필요가 없다.

csv 파일로 덤프하기 전에 마지막 수정을 한번 하자.  
purchase_date와 purchase_month는 가까이 있는 편이 다루기 편하다. 따라서 칼럼의 배치를 조정한 후 파일을 저장하여 나중에 데이터를 봤을 때 직관적으로 이해하기 쉽도록 하자.

In [102]:
join_data.columns

Index(['purchase_date', 'item_name', 'item_price', 'purchase_month', '고객이름',
       '지역', '등록일', '등록연월'],
      dtype='object')

In [105]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '고객이름', '지역', '등록일', '등록연월']]
dump_data.head()

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07,201706


In [106]:
#같은 폴더에 dump_data가 출력된 것 확인 가능하다.
dump_data.to_csv("dump_data.csv", index = False)

# 데이터를 집계하자

우선 덤프 파일을 읽어들이자.


In [110]:
uriage_data = pd.read_csv("uriage.csv")

FileNotFoundError: ignored

In [107]:
#이건 왜 되지?
import_data = pd.read_csv("dump_data.csv")

purchase_month를 세로축으로 해서 상품별로 집계해보자.

In [113]:
byItem = import_data.pivot_table(index = "purchase_month", columns = 'item_name', aggfunc = 'size', fill_value = 0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


계속해서 purchase_month를 세로축으로 지정하고, 매출 금액, 고객, 지역을 집계해보자.

In [114]:
byPrice = import_data.pivot_table(index = "purchase_month", columns = 'item_name', values = 'item_price', aggfunc = 'sum', fill_value = 0)
byPrice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [116]:
byConsumer = import_data.pivot_table(index = "purchase_month", columns = '고객이름', aggfunc = 'size', fill_value = 0)
byConsumer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,...,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,...,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,...,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,...,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,...,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,...,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,...,2,1,0,4,0,2,6,1,2,3


In [117]:
byRegion = import_data.pivot_table(index = "purchase_month", columns = '지역', aggfunc = 'size', fill_value = 0)
byRegion

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


마지막으로, 집계 기간에 구매 이력이 없는 사용자를 확인해보자.  
집계 기간 내에 구매하지 않은 고객은 purchase_date 등이 NaN으로 채워진다. 이것을 조건으로 해서 데이터를 추출하면 집계 기간에 구매 이력이 없는 고객을 조사할 수 있다.

In [121]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on = "customer_name", right_on = "고객이름", how = "right")
away_data[away_data['purchase_date'].isnull()][['고객이름', '등록일']]

,고객이름,등록일
2999,김서우,2019-04-23
